In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")
os.chdir("drive/MyDrive/traffic-forecasting-GNN")

In [ ]:
!pip install torch-geometric

In [ ]:
from traffic_data import METRLADatasetLoader
import models as mod
import utils
import torch
import matplotlib.pyplot as plt
import pandas as pd
from itertools import combinations

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# get the data loader for the data
# num_timesteps_in=12, num_timesteps_out=1, train_p=0.8, batch_size=256
train_loader, test_loader, train_loader_full, test_loader_full, adj, mean, std = utils.get_data_loaders()

X_test, Y_test = next(iter(test_loader_full))
criterion = torch.nn.MSELoss()
Y_test = Y_test.to(device)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
# Pasting

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original sin reemplazo)
train_loader_pasting, _, _, test_loader_full_pasting, _, _, _ = utils.get_data_loaders(pasting=True, seed=1) # semilla 1 para gat + lstm
# en este subset se entrenara el gat + lstm
model = mod.GATv2_LSTM_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_pasting,
                                                          test_loader=test_loader_full_pasting,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_pasting/gat_lstm.pt")
torch.save(outputs, "trained_models_pasting/gat_lstm_outputs.pt")
lossi_gat_lstm_pasting = lossi
lossi_test_gat_lstm_pasting = lossi_test
xs_gat_lstm_pasting = xs

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original sin reemplazo)
train_loader_pasting, _, _, test_loader_full_pasting, _, _, _ = utils.get_data_loaders(pasting=True, seed=2) # semilla 2 para gat + gru
# en este subset se entrenara el gat + gru
model = mod.GATv2_GRU_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_pasting,
                                                          test_loader=test_loader_full_pasting,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_pasting/gat_gru.pt")
torch.save(outputs, "trained_models_pasting/gat_gru_outputs.pt")
lossi_gat_gru_pasting = lossi
lossi_test_gat_gru_pasting = lossi_test
xs_gat_gru_pasting = xs

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original sin reemplazo)
train_loader_pasting, _, _, test_loader_full_pasting, _, _, _ = utils.get_data_loaders(pasting=True, seed=3) # semilla 3 para gcn + lstm
# en este subset se entrenara el gcn + lstm
model = mod.GCN_LSTM_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_pasting,
                                                          test_loader=test_loader_full_pasting,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_pasting/gcn_lstm.pt")
torch.save(outputs, "trained_models_pasting/gcn_lstm_outputs.pt")
lossi_gcn_lstm_pasting = lossi
lossi_test_gcn_lstm_pasting = lossi_test
xs_gcn_lstm_pasting = xs

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original sin reemplazo)
train_loader_pasting, _, _, test_loader_full_pasting, _, _, _ = utils.get_data_loaders(pasting=True, seed=4) # semilla 4 para gcn + gru
# en este subset se entrenara el gcn + gru
model = mod.GCN_GRU_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_pasting,
                                                          test_loader=test_loader_full_pasting,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_pasting/gcn_gru.pt")
torch.save(outputs, "trained_models_pasting/gcn_gru_outputs.pt")
lossi_gcn_gru_pasting = lossi
lossi_test_gcn_gru_pasting = lossi_test
xs_gcn_gru_pasting = xs

In [ ]:
# Crear una figura y una matriz de subgráficos
fig, axs = plt.subplots(2, 2, figsize=(10, 5))

# Lista de datos y títulos para iterar
modelos = [
    (lossi_gat_lstm_pasting, lossi_test_gat_lstm_pasting, xs_gat_lstm_pasting, "GAT_LSTM"),
    (lossi_gat_gru_pasting, lossi_test_gat_gru_pasting, xs_gat_gru_pasting, "GAT_GRU"),
    (lossi_gcn_lstm_pasting, lossi_test_gcn_lstm_pasting, xs_gcn_lstm_pasting, "GCN_LSTM"),
    (lossi_gcn_gru_pasting, lossi_test_gcn_gru_pasting, xs_gcn_gru_pasting, "GCN_GRU")
]

# Lista para almacenar las líneas de los gráficos para la leyenda
lines = []
labels = []

# Iterar sobre los subgráficos y los datos
for i, (ax, (lossi, lossi_test, xs, title)) in enumerate(zip(axs.flat, modelos)):
    line1, = ax.plot(lossi, label="Error en el conjunto de entrenamiento")
    line2, = ax.plot(xs, lossi_test, label="Error en el conjunto de testeo")
    line3, = ax.plot(utils.moving_average(lossi, 500), label="Tendencia error", lw=3, c='#1a5989')
    ax.set_title(title)

    # Condición para asignar etiquetas de los ejes
    if i // 2 == 1:  # Filas inferiores (índices 2 y 3)
        ax.set_xlabel('Iteraciones de entrenamiento')
    if i % 2 == 0:  # Columnas izquierdas (índices 0 y 2)
        ax.set_ylabel('Pérdida')

    # Solo agregar las líneas y etiquetas una vez
    if not lines:
        lines.extend([line1, line2, line3])
        labels.extend([line1.get_label(), line2.get_label(), line3.get_label()])

# Escribir titulo
fig.suptitle("Evolución de las pérdidas de cada modelo USANDO PASTING")

# Ajustar el layout para que no haya superposición de elementos
plt.tight_layout()

# Crear una leyenda global
fig.legend(lines, labels, loc='center left', bbox_to_anchor=(0.86, 0.5))

# Ajustar los márgenes para hacer espacio para la leyenda a la derecha
plt.subplots_adjust(right=0.85)

# Mostrar la figura completa con los 4 subgráficos
plt.show()

In [ ]:
# Promedio de los modelos

In [ ]:
outputs_gat_lstm = torch.load("trained_models_pasting/gat_lstm_outputs.pt")
outputs_gat_gru = torch.load("trained_models_pasting/gat_gru_outputs.pt")
outputs_gcn_lstm = torch.load("trained_models_pasting/gcn_lstm_outputs.pt")
outputs_gcn_gru = torch.load("trained_models_pasting/gcn_gru_outputs.pt")

In [ ]:
def generar_combinaciones(lista):
    combinaciones = []
    for i in range(1, len(lista) + 1):
        combinaciones.extend(combinations(lista, i))
    return combinaciones

outputs_dict = {
    'GAT + LSTM': outputs_gat_lstm,
    'GCN + LSTM': outputs_gcn_lstm,
    'GAT + GRU': outputs_gat_gru,
    'GCN + GRU': outputs_gcn_gru
}

lista = ['GAT + LSTM', 'GCN + LSTM', 'GAT + GRU', 'GCN + GRU']
combinaciones = generar_combinaciones(lista)

In [ ]:
res = {
    'Modelos': [],
    'Test loss': []
}

for combinacion in combinaciones:
    full_predicciones = outputs_dict[combinacion[0]]
    for nombre_modelo in combinacion[1:]:
        full_predicciones = torch.cat((full_predicciones, outputs_dict[nombre_modelo]), dim=2)
    output_ensamble_media = torch.mean(full_predicciones, dim=2, keepdim=True)
    test_loss = criterion(output_ensamble_media, Y_test).item()
    res['Modelos'].append("PASTING_" + str(combinacion))
    res['Test loss'].append(test_loss)

In [ ]:
df = pd.DataFrame(res)
df.sort_values("Test loss")

In [ ]:
# Bagging

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original CON reemplazo)
train_loader_bagging, _, _, test_loader_full_bagging, _, _, _ = utils.get_data_loaders(bagging=True, seed=5) # semilla 5 para gat + lstm
# en este subset se entrenara el gat + lstm
model = mod.GATv2_LSTM_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_bagging,
                                                          test_loader=test_loader_full_bagging,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_bagging/gat_lstm.pt")
torch.save(outputs, "trained_models_bagging/gat_lstm_outputs.pt")
lossi_gat_lstm_bagging = lossi
lossi_test_gat_lstm_bagging = lossi_test
xs_gat_lstm_bagging = xs

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original sin reemplazo)
train_loader_bagging, _, _, test_loader_full_bagging, _, _, _ = utils.get_data_loaders(bagging=True, seed=6) # semilla 6 para gat + gru
# en este subset se entrenara el gat + gru
model = mod.GATv2_GRU_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_bagging,
                                                          test_loader=test_loader_full_bagging,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_bagging/gat_gru.pt")
torch.save(outputs, "trained_models_bagging/gat_gru_outputs.pt")
lossi_gat_gru_bagging = lossi
lossi_test_gat_gru_bagging = lossi_test
xs_gat_gru_bagging = xs

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original sin reemplazo)
train_loader_bagging, _, _, test_loader_full_bagging, _, _, _ = utils.get_data_loaders(bagging=True, seed=7) # semilla 7 para gcn + lstm
# en este subset se entrenara el gcn + lstm
model = mod.GCN_LSTM_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_bagging,
                                                          test_loader=test_loader_full_bagging,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_bagging/gcn_lstm.pt")
torch.save(outputs, "trained_models_bagging/gcn_lstm_outputs.pt")
lossi_gcn_lstm_bagging = lossi
lossi_test_gcn_lstm_bagging = lossi_test
xs_gcn_lstm_bagging = xs

In [ ]:
# particionamos la base para entrenar un modelo (66% de la base original sin reemplazo)
train_loader_bagging, _, _, test_loader_full_bagging, _, _, _ = utils.get_data_loaders(bagging=True, seed=8) # semilla 8 para gcn + gru
# en este subset se entrenara el gcn + gru
model = mod.GCN_GRU_Model()
model, lossi, lossi_test, xs, outputs = utils.train_model(model=model,
                                                          device=device,
                                                          num_epochs=50,
                                                          train_loader=train_loader_bagging,
                                                          test_loader=test_loader_full_bagging,
                                                          adj=adj,
                                                          lr=0.0001)
# guardar resultados
torch.save(model, "trained_models_bagging/gcn_gru.pt")
torch.save(outputs, "trained_models_bagging/gcn_gru_outputs.pt")
lossi_gcn_gru_bagging = lossi
lossi_test_gcn_gru_bagging = lossi_test
xs_gcn_gru_bagging = xs

In [ ]:
# Crear una figura y una matriz de subgráficos
fig, axs = plt.subplots(2, 2, figsize=(10, 5))

# Lista de datos y títulos para iterar
modelos = [
    (lossi_gat_lstm_bagging, lossi_test_gat_lstm_bagging, xs_gat_lstm_bagging, "GAT_LSTM"),
    (lossi_gat_gru_bagging, lossi_test_gat_gru_bagging, xs_gat_gru_bagging, "GAT_GRU"),
    (lossi_gcn_lstm_bagging, lossi_test_gcn_lstm_bagging, xs_gcn_lstm_bagging, "GCN_LSTM"),
    (lossi_gcn_gru_bagging, lossi_test_gcn_gru_bagging, xs_gcn_gru_bagging, "GCN_GRU")
]

# Lista para almacenar las líneas de los gráficos para la leyenda
lines = []
labels = []

# Iterar sobre los subgráficos y los datos
for i, (ax, (lossi, lossi_test, xs, title)) in enumerate(zip(axs.flat, modelos)):
    line1, = ax.plot(lossi, label="Error en el conjunto de entrenamiento")
    line2, = ax.plot(xs, lossi_test, label="Error en el conjunto de testeo")
    line3, = ax.plot(utils.moving_average(lossi, 500), label="Tendencia error", lw=3, c='#1a5989')
    ax.set_title(title)

    # Condición para asignar etiquetas de los ejes
    if i // 2 == 1:  # Filas inferiores (índices 2 y 3)
        ax.set_xlabel('Iteraciones de entrenamiento')
    if i % 2 == 0:  # Columnas izquierdas (índices 0 y 2)
        ax.set_ylabel('Pérdida')

    # Solo agregar las líneas y etiquetas una vez
    if not lines:
        lines.extend([line1, line2, line3])
        labels.extend([line1.get_label(), line2.get_label(), line3.get_label()])

# Escribir titulo
fig.suptitle("Evolución de las pérdidas de cada modelo USANDO BAGGING")

# Ajustar el layout para que no haya superposición de elementos
plt.tight_layout()

# Crear una leyenda global
fig.legend(lines, labels, loc='center left', bbox_to_anchor=(0.86, 0.5))

# Ajustar los márgenes para hacer espacio para la leyenda a la derecha
plt.subplots_adjust(right=0.85)

# Mostrar la figura completa con los 4 subgráficos
plt.show()

In [ ]:
# Promedio de los modelos

In [ ]:
outputs_gat_lstm = torch.load("trained_models_bagging/gat_lstm_outputs.pt")
outputs_gat_gru = torch.load("trained_models_bagging/gat_gru_outputs.pt")
outputs_gcn_lstm = torch.load("trained_models_bagging/gcn_lstm_outputs.pt")
outputs_gcn_gru = torch.load("trained_models_bagging/gcn_gru_outputs.pt")

In [ ]:
outputs_dict = {
    'GAT + LSTM': outputs_gat_lstm,
    'GCN + LSTM': outputs_gcn_lstm,
    'GAT + GRU': outputs_gat_gru,
    'GCN + GRU': outputs_gcn_gru
}

for combinacion in combinaciones:
    full_predicciones = outputs_dict[combinacion[0]]
    for nombre_modelo in combinacion[1:]:
        full_predicciones = torch.cat((full_predicciones, outputs_dict[nombre_modelo]), dim=2)
    output_ensamble_media = torch.mean(full_predicciones, dim=2, keepdim=True)
    test_loss = criterion(output_ensamble_media, Y_test).item()
    res['Modelos'].append("BAGGING_" + str(combinacion))
    res['Test loss'].append(test_loss)

In [ ]:
df = pd.DataFrame(res)
df.sort_values("Test loss")

In [ ]:
df.to_excel("Resultados_bag_past.xlsx")